In [137]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datefilter='2024-01-01'

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()

def hist_data_dog(surface, sex, odds_low,odds_high,date):
    data=pd.read_sql_query(f"Select distinct * from results_{surface}_1 where date > '{date}' ",con=devengine)#.to_clipboard(index=False)
    data["Fav_Odds"] = data.Fav_Odds.astype(float)
    data["Dog_Odds"] = data.Dog_Odds.astype(float)
    data['Fav_Win'] = data['Winner']==data['Fav']
    data['DogWin'] = data['Winner']!=data['Fav']
    data=data[(data['Fav_Odds']>=odds_low)&(data['Fav_Odds']<=odds_high)]
    data=data[data['Sex']==sex]
    return data
hard_womens=hist_data_dog('HArd','Womens',1,2,datefilter)
hard_mens=hist_data_dog('HArd','Mens',1,2,datefilter)
temp=pd.concat([hard_womens,hard_mens]).sort_values(by='Date')
temp = temp.reset_index(drop=True)
temp['Fav_Loss'] = (temp['Fav_Win'] == 0).astype(int)
temp['Fav_Loss_Count'] = temp.groupby('Fav')['Fav_Loss'].cumsum()
temp['Dog_Win'] = (temp['Fav_Win'] == 0).astype(int)
temp['Dog_Win_Count'] = temp.groupby('Dog')['Dog_Win'].cumsum()

In [138]:
#Female History
date_filter=temp[temp['Date']>'2025-01-01']
oddsfilter=date_filter[date_filter['Fav_Odds']>1.6].copy()
#oddsfilter=date_filter[date_filter['Dog_Odds']>=2.1]
oddsfilter=oddsfilter[oddsfilter['Dog_Win_Count']>0]
oddsfilter=oddsfilter[oddsfilter['Fav_Loss_Count']>0]
#oddsfilter=oddsfilter[oddsfilter['Elo_Fav']==oddsfilter['Fav']]
#oddsfilter=oddsfilter[oddsfilter['Dog_Top100']>.5]
#oddsfilter=oddsfilter[oddsfilter['Fav_Top100']<.5]
#oddsfilter=oddsfilter[oddsfilter['dog_rank']<200]
#oddsfilter=oddsfilter[oddsfilter['fav_rank']<300]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Womens']
len(oddsfilter[oddsfilter['Fav_Loss']==1])/len(oddsfilter),len(oddsfilter)
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter.tail(5)

Total Profit: 5481.0 289 0.5709342560553633


,Date,Winner_Odds,Winner,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Games,Dog,Dog_Games,...,Dog_Top100,fav_rank_high,dog_rank_high,Fav_Win,DogWin,Fav_Loss,Fav_Loss_Count,Dog_Win,Dog_Win_Count,Profit
7939,2025-03-28,1.95,Rebeka Masarova,1.80,1.95,Bernarda Pera,Bernarda Pera,11,Rebeka Masarova,27,...,0.5,27.0,62.0,False,True,1,5,1,9,95.0
7950,2025-03-29,2.08,Heather Watson,1.71,2.08,Marina Stakusic,Heather Watson,11,Heather Watson,14,...,0.4,114.0,38.0,False,True,1,5,1,3,108.0
7951,2025-03-29,2.14,Linda Fruhvirtova,1.66,2.14,Maya Joint,Maya Joint,10,Linda Fruhvirtova,24,...,0.3,80.0,49.0,False,True,1,6,1,5,114.0
7959,2025-03-30,1.75,Jaqueline Cristian,1.75,2.01,Jaqueline Cristian,Jaqueline Cristian,17,Rebeka Masarova,27,...,0.5,57.0,62.0,True,False,0,2,0,9,-100.0
7962,2025-03-31,1.67,Jaqueline Cristian,1.67,2.16,Jaqueline Cristian,Jaqueline Cristian,14,Linda Fruhvirtova,22,...,0.3,57.0,49.0,True,False,0,2,0,5,-100.0


In [139]:
#Male History
date_filter=temp[temp['Date']>'2024-01-01']
oddsfilter=date_filter[date_filter['Fav_Odds']>1.7].copy()
#oddsfilter=oddsfilter[oddsfilter['Dog_Odds']<2]
#oddsfilter=oddsfilter[oddsfilter['Fav_Odds']<1.8].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Win_Count']>0]
oddsfilter=oddsfilter[oddsfilter['Fav_Loss_Count']>0]
oddsfilter=oddsfilter[oddsfilter['Elo_Fav']!=oddsfilter['Dog']]
#oddsfilter=oddsfilter[oddsfilter['Dog_Top100']>.5]
#oddsfilter=oddsfilter[oddsfilter['Fav_Top100']<.5]
#oddsfilter=oddsfilter[oddsfilter['dog_rank']>200]
#oddsfilter=oddsfilter[oddsfilter['fav_rank']>100]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Mens']
len(oddsfilter[oddsfilter['Fav_Loss']==1])/len(oddsfilter),len(oddsfilter)
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter.tail(5)

Total Profit: 8663.0 480 0.59375


,Date,Winner_Odds,Winner,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Games,Dog,Dog_Games,...,Dog_Top100,fav_rank_high,dog_rank_high,Fav_Win,DogWin,Fav_Loss,Fav_Loss_Count,Dog_Win,Dog_Win_Count,Profit
8098,2025-04-27,1.81,Aziz Dougaz,1.81,1.93,Aziz Dougaz,Aziz Dougaz,15,Michael Geerts,12,...,1.0,185.0,212.0,True,False,0,5,0,9,-100.0
8127,2025-04-30,1.75,Hsiou Hsu Yu,1.75,1.99,Hsiou Hsu Yu,Hsiou Hsu Yu,19,Antoine Escoffier,21,...,0.4,158.0,135.0,True,False,0,11,0,11,-100.0
8144,2025-05-04,2.01,Terence Atmane,1.73,2.01,Tristan Schoolkate,Tristan Schoolkate,32,Terence Atmane,31,...,0.4,119.0,118.0,False,True,1,14,1,12,101.0
8155,2025-05-05,1.94,Evan Zhu,1.77,1.94,Jake Delaney,Jake Delaney,10,Evan Zhu,11,...,0.0,434.0,271.0,False,True,1,1,1,4,94.0
8167,2025-05-08,1.75,Ilia Simakin,1.75,1.98,Ilia Simakin,Ilia Simakin,11,Alibek Kachmazov,17,...,0.5,264.0,161.0,True,False,0,1,0,12,-100.0


In [140]:
today=pd.read_pickle('.\Hard_Today')
today=today[today['Fav_Odds']>1.6]
today=today[today['Dog_Win_Count']>2]
today=today[today['Fav_Loss_Count']>2]
today[today['Sex']=='Womens']

<>:1: SyntaxWarning: invalid escape sequence '\H'
<>:1: SyntaxWarning: invalid escape sequence '\H'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_10668\3636087751.py:1: SyntaxWarning: invalid escape sequence '\H'
  today=pd.read_pickle('.\Hard_Today')


,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,...,Fav_Top100,Dog_Top100,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%,fav_last_five_win_perc,dog_last_five_win_perc,Fav_Loss_Count,Dog_Win_Count


In [141]:
lowodds=pd.read_pickle(r'.\Hard_Today')
lowodds=lowodds[lowodds['Fav_Odds']<1.3].copy()
lowodds=lowodds[lowodds['Fav_Odds']>1]
favs=pd.read_csv('Fav_Hard.csv')
fav_list=favs['Name'].tolist()
lowodds=lowodds[(lowodds['Fav']).isin(fav_list)]
lowodds=lowodds[lowodds['dog_rank']>50]
lowodds

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,...,Fav_Top100,Dog_Top100,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%,fav_last_five_win_perc,dog_last_five_win_perc,Fav_Loss_Count,Dog_Win_Count


In [142]:
today=pd.read_pickle('.\Hard_Today')
today=today[today['Fav_Odds']>1.7]
today=today[today['Fav_Loss_Count']>2]
today=today[today['Dog_Win_Count']>2]
#today=today[today['Elo_Fav']==today['Dog']]
#today=today[today['fav_rank']>100]
#today=today[today['dog_rank']>200]
today[today['Sex']!='Womens']

<>:1: SyntaxWarning: invalid escape sequence '\H'
<>:1: SyntaxWarning: invalid escape sequence '\H'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_10668\2673991922.py:1: SyntaxWarning: invalid escape sequence '\H'
  today=pd.read_pickle('.\Hard_Today')


,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,...,Fav_Top100,Dog_Top100,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%,fav_last_five_win_perc,dog_last_five_win_perc,Fav_Loss_Count,Dog_Win_Count


In [143]:
# Current date and time
current_datetime = datetime.datetime.now()#+ datetime.timedelta(days=-1)
# Convert to string
date_string = current_datetime.strftime("%Y-%m-%d")

today=pd.read_pickle('.\Hard_Today')
today=today[today['Fav_Odds']>1.7]
today=today[today['Dog_Win_Count']>2]
today=today[today['Fav_Loss_Count']>2]
today['Date']=date_string
today.to_sql('Preditions_Hard',con=devengine,if_exists='append')

<>:6: SyntaxWarning: invalid escape sequence '\H'
<>:6: SyntaxWarning: invalid escape sequence '\H'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_10668\3208968739.py:6: SyntaxWarning: invalid escape sequence '\H'
  today=pd.read_pickle('.\Hard_Today')


0

In [144]:
predictions=pd.read_sql_table('Preditions_Hard',con=devengine)
results=temp
merged_df = pd.merge(
    predictions, 
    results[['Date', 'Fav', 'Dog', 'Winner']],  # Select only the Winner column from results
    on=['Date', 'Fav', 'Dog'], 
    how='left'  # Use left join to retain all rows from predictions
)
#merged_df_filtered=merged_df[merged_df['Dog_Win_Count']>2].copy()
#merged_df_filtered=merged_df_filtered[merged_df_filtered['Fav_Loss_Count']>2]
#merged_df_filtered=merged_df_filtered[merged_df_filtered['Fav_Odds']>1.7]

merged_df_filtered=merged_df[merged_df['Winner'].notna()].copy()
merged_df_filtered=merged_df_filtered[merged_df_filtered['Dog_Odds']>=2.1]
#merged_df_filtered=merged_df_filtered[merged_df_filtered['Date']=='2025-02-20']
merged_df_filtered['Profit'] = merged_df_filtered.apply(
    lambda row: (row['Dog_Odds'] * stake)-stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = merged_df_filtered['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(merged_df_filtered),len(merged_df_filtered[merged_df_filtered['Dog']==merged_df_filtered['Winner']])/len(merged_df_filtered))
merged_df_filtered


Total Profit: 248.0 6 0.6666666666666666


,index,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,...,Fav_Return%,Dog_Serve%,Dog_Return%,fav_last_five_win_perc,dog_last_five_win_perc,Fav_Loss_Count,Dog_Win_Count,Date,Winner,Profit
2,19,05:20,1.71,2.13,Claire Liu,Claire Liu,100%,2,Linda Fruhvirtova,67%,...,0.37,0.66,0.35,0.8,0.4,8.0,7.0,2025-03-19,Linda Fruhvirtova,113.0
11,19,05:30,1.71,2.12,Ons Jabeur,Katerina Siniakova,58%,31,Katerina Siniakova,48%,...,0.40,0.60,0.36,0.4,0.6,9.0,12.0,2025-03-21,Ons Jabeur,-100.0
12,28,09:45,1.72,2.12,Beatriz Haddad Maia,Beatriz Haddad Maia,50%,2,Linda Fruhvirtova,33%,...,0.31,0.66,0.35,0.0,0.6,16.0,8.0,2025-03-21,Linda Fruhvirtova,112.0
15,13,05:35,1.71,2.12,Linda Noskova,McCartney Kessler,56%,43,McCartney Kessler,38%,...,0.26,NaN,NaN,0.6,0.6,11.0,4.0,2025-03-22,McCartney Kessler,112.0
18,20,09:10,1.73,2.11,Alejandro Davidovich Fokina,Alejandro Davidovich Fokina,45%,44,Frances Tiafoe,45%,...,0.26,0.81,0.19,0.6,0.4,14.0,7.0,2025-03-23,Frances Tiafoe,111.0
24,1,02:15,1.71,2.14,Sebastian Korda,Gael Monfils,56%,41,Gael Monfils,63%,...,0.26,0.84,0.28,0.4,0.8,20.0,9.0,2025-03-26,Sebastian Korda,-100.0
